In [188]:
from IPython.display import IFrame
IFrame(src='https://www.abuseat.org/public/asninfections.html', width=1100, height=600)

In [191]:
#!/usr/bin/env python
#https://www.abuseat.org/public/asninfections.html
import requests
import pandas as pd
import json



asn = '45271'

ip_ranges = requests.get('https://asn.ipinfo.app/api/text/list/' +asn)
data = ip_ranges.content
data = data.split('\n')
df = pd.DataFrame(data)
df = df[~df[0].str.contains('::')]
df = df[df[0].str.contains('/')]
pd.set_option('display.max_rows', None)
cidr = df[0]
cidr

0         106.76.22.0/24
1         27.97.152.0/22
2         106.78.33.0/24
6          27.97.49.0/24
7         1.187.142.0/23
8        106.66.214.0/23
10        106.76.77.0/24
13       106.76.197.0/24
17       106.66.206.0/23
18       106.66.131.0/24
23        106.78.88.0/24
27       112.110.78.0/24
29        27.97.232.0/22
32          27.97.0.0/20
34        27.97.224.0/23
36       106.79.216.0/21
39        106.78.51.0/24
40       106.76.209.0/24
42        106.76.17.0/24
43        49.15.210.0/24
44        49.15.175.0/24
51      112.110.247.0/24
54        106.76.40.0/24
60       106.76.244.0/22
65        27.97.144.0/22
66        49.14.101.0/24
68        49.15.183.0/24
69       106.66.194.0/23
72         49.14.64.0/22
76         106.78.2.0/23
78        106.66.56.0/22
81       106.66.254.0/23
82         49.14.97.0/24
84      112.110.249.0/24
90        106.78.87.0/24
91       106.66.146.0/24
96       106.77.136.0/22
97        27.97.234.0/24
98        27.97.199.0/24
99        27.97.208.0/22


In [192]:
import csv
from netaddr import IPAddress 
from netaddr import IPNetwork
for networks in cidr:
    ip = IPNetwork(networks)
    subnets = list(ip.subnet(24))
    #print subnets
    count = 0
    
    while count < len(subnets):
        #print(subnets[count])
        #count +=1
        with open(r'/Users/johnsmith/notebooks/'+asn+'.csv', "ab") as f:
            writer = csv.writer(f)
            #writer = csv.writer(open("docidrs.csv", "wb"),delimiter='\n')
            writer.writerow([subnets[count]])
            #print subnets[count]
            count +=1

In [193]:
asnData = pd.read_csv(r'/Users/johnsmith/notebooks/'+asn+'.csv',names="A")
asnCidr = asnData.loc[:,'A'].str.rsplit(pat='.',n=1,expand=True)[0] # < -- Count by /24 CIDR block
#asnCidr

In [194]:
from parsebrologs import ParseBroLogs
data = ParseBroLogs('conn.log')
df = pd.DataFrame(json.loads(data.to_json()))
#df
#server = df[~df['id.resp_h'].str.contains(':')]
#addr = server['id.resp_h']
cidr = df.loc[:,'id.resp_h'].str.rsplit(pat='.',n=1,expand=True)[0] # < -- Count by /24 CIDR block
#cidr.value_counts().head(50)

In [197]:
joined_ips=pd.merge(df,asnData, right_on=asnCidr, left_on=cidr)
joined_df = joined_ips.drop_duplicates()
#joined_df
#joined_df.groupby(['id.orig_h','key_0']).size()